In [43]:
!git clone https://github.com/m0chi1216/hlab2023-nlp100.git

fatal: destination path 'hlab2023-nlp100' already exists and is not an empty directory.


In [44]:
cd hlab2023-nlp100

c:\Users\rio_m\OneDrive - 国立大学法人東海国立大学機構\100knock\hlab2023-nlp100\hlab2023-nlp100


In [45]:
!git checkout -b suzuki origin/suzuki

fatal: a branch named 'suzuki' already exists


In [46]:
cd c:\Users\rio_m\OneDrive - 国立大学法人東海国立大学機構\100knock\hlab2023-nlp100

c:\Users\rio_m\OneDrive - 国立大学法人東海国立大学機構\100knock\hlab2023-nlp100


In [47]:
with open("readme.txt","r")as fp:
  words=fp.read()
  print(words)
  fp.close()

SUMMARY: Dataset of references (urls) to news web pages

DESCRIPTION: Dataset of references to news web pages collected from an online aggregator in the period from March 10 to August 10 of 2014. The resources are grouped into clusters that represent pages discussing the same news story. The dataset includes also references to web pages that point (has a link to) one of the news page in the collection.

TAGS: web pages, news, aggregator, classification, clustering

LICENSE: Public domain - Due to restrictions on content and use of the news sources, the corpus is limited to web references (urls) to web pages and does not include any text content. The references have been retrieved from the news aggregator through traditional web browsers. 

FILE ENCODING: UTF-8

FORMAT: Tab delimited CSV files. 

DATA SHAPE AND STATS: 422937 news pages and divided up into:

152746 	news of business category
108465 	news of science and technology category
115920 	news of business category
 45615 	news of

In [48]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [49]:
%pip install scikit-learn 

In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('./newsCorpora.csv',header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])
df=df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']),['TITLE', 'CATEGORY']]

train,valid=train_test_split(df,test_size=0.2,random_state=1,shuffle=True,stratify=df["CATEGORY"])
valid,test=train_test_split(valid,test_size=0.5,random_state=1,shuffle=True,stratify=valid["CATEGORY"])

train.to_csv('./train.txt', sep='\t', index=False)
valid.to_csv('./valid.txt', sep='\t', index=False)
test.to_csv('./test.txt', sep='\t', index=False)

print('学習データ')
print(train['CATEGORY'].value_counts())
print('検証データ')
print(valid['CATEGORY'].value_counts())
print('評価データ')
print(test['CATEGORY'].value_counts())

学習データ
CATEGORY
b    4502
e    4223
t    1219
m     728
Name: count, dtype: int64
検証データ
CATEGORY
b    563
e    528
t    152
m     91
Name: count, dtype: int64
評価データ
CATEGORY
b    562
e    528
t    153
m     91
Name: count, dtype: int64


In [51]:
import string
import re

def preprocessing(text):
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  text = text.translate(table)  # 記号をスペースに置換
  text = text.lower()  # 小文字化
  text = re.sub('[0-9]+', '0', text)  # 数字列を0に置換

  return text

In [52]:
df=pd.concat([train,valid,test])
df.reset_index(drop=True,inplace=True)
df["TITLE"]=df["TITLE"].map(lambda x:preprocessing(x))
print(df.head())

                                               TITLE CATEGORY
0  corrected update 0 brent crude falls again as ...        b
1  corrected retailer michaels stores confirms pa...        b
2  britney spears hits her local supermarket dres...        e
3  orders propel us service industries as sales i...        b
4  column fed to widen main st wall st gap  james...        b


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_valid = df[:len(train) + len(valid)]
test = df[len(train) + len(valid):]

vec_tfidf=TfidfVectorizer(min_df=30)
X_train_valid = vec_tfidf.fit_transform(train_valid['TITLE']) 
X_test = vec_tfidf.transform(test['TITLE'])

X_train_valid = pd.DataFrame(X_train_valid.toarray(),columns=vec_tfidf.get_feature_names_out())
X_test = pd.DataFrame(X_test.toarray(), columns=vec_tfidf.get_feature_names_out())

X_train=X_train_valid[:len(train)]
X_valid=X_train_valid[len(train):]

print(X_train.head())


   0th  about  above  action  actor  adds  affirms  africa     after    again  \
0  0.0    0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.000000  0.33371   
1  0.0    0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.000000  0.00000   
2  0.0    0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.425078  0.00000   
3  0.0    0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.000000  0.00000   
4  0.0    0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.000000  0.00000   

   ...  yellen  yen  yet  yields  york  you  young  your  zac  zone  
0  ...     0.0  0.0  0.0     0.0   0.0  0.0    0.0   0.0  0.0   0.0  
1  ...     0.0  0.0  0.0     0.0   0.0  0.0    0.0   0.0  0.0   0.0  
2  ...     0.0  0.0  0.0     0.0   0.0  0.0    0.0   0.0  0.0   0.0  
3  ...     0.0  0.0  0.0     0.0   0.0  0.0    0.0   0.0  0.0   0.0  
4  ...     0.0  0.0  0.0     0.0   0.0  0.0    0.0   0.0  0.0   0.0  

[5 rows x 682 columns]


In [54]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(random_state=123, max_iter=10000)
lg.fit(X_train, train['CATEGORY'])

LogisticRegression(max_iter=10000, random_state=123)

In [55]:
import numpy as np
def score(lg,x):
  return[np.max(lg.predict_proba(x),axis=1),lg.predict(x)]

In [56]:
train_predict=score(lg,X_train)
test_predict=score(lg,X_test)
print(train_predict)

[array([0.94465832, 0.53412461, 0.74239951, ..., 0.96803602, 0.66861914,
       0.92907688]), array(['b', 'b', 'e', ..., 'b', 'b', 'b'], dtype=object)]


In [57]:
from sklearn.metrics import accuracy_score

train_accuracy=accuracy_score(train["CATEGORY"],train_predict[1])
test_accuracy=accuracy_score(test["CATEGORY"],test_predict[1])

print(f"学習データの正解率={train_accuracy:.3f}")
print(f"テストデータの正解率={test_accuracy:.3f}")

学習データの正解率=0.868
テストデータの正解率=0.827


In [58]:
!git status

On branch suzuki
Your branch is up to date with 'origin/suzuki'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)
	deleted:    100knock_2.ipynb
	modified:   hlab2023-nlp100 (untracked content)

no changes added to commit (use "git add" and/or "git commit -a")


In [59]:
!git config --global user.email "riors2020@gmail.com"
!git config --global user.name "rio-user11"

In [60]:
!git add .

In [61]:
!git commit -m “commit2”


[suzuki 1786487] “commit2”
 1 file changed, 627 deletions(-)
 delete mode 100644 100knock_2.ipynb


In [62]:
!git push origin suzuki

remote: Resolving deltas:   0% (0/1)        
remote: Resolving deltas: 100% (1/1)        
remote: Resolving deltas: 100% (1/1), completed with 1 local object.        
To https://github.com/m0chi1216/hlab2023-nlp100.git
   3eae126..1786487  suzuki -> suzuki
